In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
from sklearn.preprocessing import LabelEncoder
import numpy as np

2023-08-17 03:33:33.995524: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-17 03:33:34.059115: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-17 03:33:34.060773: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 03:33:35.204783: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
train_file = '/home/azureuser/projects/aerius/ai/myapp/chatbot/datas/intent2.csv'
data = pd.read_csv(train_file, delimiter = ',')
queries = data['query'].tolist()
intents = data['intent'].tolist()

print(data.head(5))
print('len of queries = ', len(queries))
print('len of intents = ', len(intents))

                              query intent
0  아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?     확인
1             실수로 취소하면 재주문해야 하는 거죠?     확인
2                     택배비 따로 추가되나요?     질문
3                          택배비 있나요?     질문
4                        택배비 따로 들어요     질문
len of queries =  209714
len of intents =  209714


In [4]:
encoder = LabelEncoder()
encoder.fit(intents)

encoded_intents = encoder.transform(intents)
num_classes = len(encoder.classes_)
print(f"The number of unique labels is {num_classes}")

The number of unique labels is 4


In [5]:
from ai.myapp.chatbot.utils.Preprocess import Preprocess
p = Preprocess()

words = []
for sentence in queries:
    if isinstance(sentence, str):  # Only process if the sentence is a string
        preprocessed = p.delete_intent_trash_tags(sentence=sentence)
        word_list, _ = p.divide_words_tags(preprocessed)
        words.extend(word_list)
    else:
        print(f"Found non-string value: {sentence}")


# Initialize the tokenizer
p.initialize_tokenizer(words)

# Convert the queries into sequences
sequences = []
for sentence in queries:
    sequence = p.text_to_sequence(sentence)
    sequences.append(sequence)

None


In [6]:
print(words[:30])
print(sequences[:20])
print(sequence)


['아침', '아니', '고', '밤', '12', '시', '30', '분', '결제', '걸', 'ㄴ데', '그렇', 'ㄴ가요', '?', '실수', '취소', '면', '재', '주문', '아야', '하', '거', '죠', '?', '택배', '비', '따로', '추가', '나요', '?']
[[1184], [], [181], [13635], [181], [], [], [13635], [], [], [], [7, 74], [], [], [19095], [1296], [506, 184, 779], [181, 13635], [], []]
[2571, 167, 6855]


In [ ]:
print(sequence[:50])

In [6]:
from ai.backend.settings import INTENT_MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=INTENT_MAX_SEQ_LEN, padding='post')

ds = tf.data.Dataset.from_tensor_slices((padded_seqs, encoded_intents))
ds = ds.shuffle(len(queries))

train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.tokenizer.word_index) + 1

input_layer = Input(shape=(INTENT_MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=INTENT_MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(filters = 128,
               kernel_size = 3,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(filters = 128,
               kernel_size = 4,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(filters = 128,
               kernel_size = 5,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(num_classes, name='logits')(dropout_hidden)
predictions = Dense(num_classes, activation=tf.nn.softmax)(logits)

model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, validation_data = val_ds, epochs = EPOCH, verbose=1)

Epoch 1/5
7340/7340 [==============================] - 168s 22ms/step - loss: 0.4791 - accuracy: 0.8469 - val_loss: 0.4328 - val_accuracy: 0.8558
Epoch 2/5
7340/7340 [==============================] - 189s 26ms/step - loss: 0.4440 - accuracy: 0.8532 - val_loss: 0.4215 - val_accuracy: 0.8563
Epoch 3/5
7340/7340 [==============================] - 203s 28ms/step - loss: 0.4329 - accuracy: 0.8574 - val_loss: 0.4102 - val_accuracy: 0.8637
Epoch 4/5
7340/7340 [==============================] - 203s 28ms/step - loss: 0.4251 - accuracy: 0.8602 - val_loss: 0.4038 - val_accuracy: 0.8662
Epoch 5/5
7340/7340 [==============================] - 206s 28ms/step - loss: 0.4191 - accuracy: 0.8622 - val_loss: 0.3924 - val_accuracy: 0.8699


In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 1. 모델 예측
y_pred = model.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)

# test_ds_resampled에서 라벨만 추출
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# 2. 성능 지표 계산
precision = precision_score(y_true, y_pred_classes, average='weighted', zero_division=1)
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
accuracy = accuracy_score(y_true, y_pred_classes)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

1049/1049 [==============================] - 13s 8ms/step
Precision: 0.7566
Recall: 0.8099
F1 Score: 0.7647
Accuracy: 0.8099


In [ ]:
loss, accuracy = model.evaluate(test_ds, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.3865000009536743
Test accuracy: 0.8722521662712097


In [ ]:
loss, accuracy = model.evaluate(test_ds, verbose=1)
print(f'Accuracy: {accuracy * 100}')
print(f'loss: {loss}')

model.save('/home/azureuser/projects/aerius/ai/myapp/chatbot/models/intent_model_original_4_labels.h5')

1049/1049 [==============================] - 4s 3ms/step - loss: 0.3890 - accuracy: 0.8698
Accuracy: 86.97725534439087
loss: 0.3890090882778168


/home/azureuser/projects/aerius/venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
